# Collecting and preparing data before analysis

In that notebook I'm loading data from one run club.

Data are being collected as usual and store into data folder.

The fodler contains *gpx* files that need to be processed such that I have Pandas Dataframe to work with after to create datavisualization.

In [2]:
!ls ../data/gpx/NDGRC*gpx | wc -l

      54


### Import a few modules

In [1]:
import sys
import numpy as np
import pandas as pd
import gpxpy
import gpxpy.gpx
import matplotlib.pyplot as plt
import geopy.distance
import glob
import os
import importlib
import matplotlib.pyplot as plt
from datetime import datetime
import time

below I'm loading my module.

In [3]:
sys.path.append("../my_modules")
import toolToReadGPX as ttrgpx

path_data     = "../data/"
path_data_csv = "../data/csv/"
run_club_name = "NDGRC" 

# Rename files

I want all files to be as follows **RunRite_year_month_day.gpx**.

I will:
+ list the files
+ lower case all the files
+ re-write **RunRite**

In [4]:
# list and rename the files
ll = glob.glob(path_data+"gpx/"+run_club_name+"*.gpx")
ll.sort()
for c, l in enumerate(ll):
    head_tail = os.path.split(ll[c])
    dst_tail  = head_tail[1].lower()
    dst_head_tail = head_tail[0]+"/"+run_club_name+"_"+dst_tail[-14:]
    os.rename(ll[c], dst_head_tail)
    
print(len(ll))

54


## Format data in pandas DataFrame

Once I found the right Python libraries, I have created some tools (ie my own modules) to load those .gpx files, to get some information about them and to save them as pandas DataFrame. The idea is to no have to re-load all my source data files each time I want to do something with them.

I'm using:
+ https://pypi.org/project/gpxpy/
+ https://pandas.pydata.org/

In [12]:
importlib.reload(ttrgpx)

# Select gpx file
list_all_files_gpx = glob.glob(path_data+"/gpx/"+run_club_name+"*.gpx")
list_all_files_gpx.sort()
print("There is {0:1.0f} files to process.".format(len(list_all_files_gpx)))

#new_list = [expression for member in iterable]
list_all_files_as_df = [ttrgpx.fun_gpx2pd(single_gpx) for single_gpx in list_all_files_gpx]

# get number of run
nb_run = len(list_all_files_as_df)

There is 54 files to process.


In [13]:
# get average distance for this year
list_cumulative_distance = [x.iloc[-1,4] for x in list_all_files_as_df]
average_run_distance     = sum(list_cumulative_distance) / (1000*len(list_cumulative_distance))

print("Average distance per run for all the gpx paths listed: {0:1.2f}km.".format(average_run_distance))

Average distance per run for all the gpx paths listed: 8.64km.


# Reduce data size

Here I will reduce the data size as I don't need so many points (eg here more than 1000) and will reduce the length to **x** points per gpx points.

And I will save the downsample path as csv files.

In [8]:
importlib.reload(ttrgpx)

# reduce the size
list_all_files_ReSample_as_df = [ttrgpx.fun_DownSample_gpx(x, number_of_sample = 200) for x in list_all_files_as_df]

# check one gpx path
list_all_files_ReSample_as_df[0].describe()

,latitude,longitude,elevation,distance,cumulative_distance
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,45.471887,-73.615784,60.395000,2.785459,4641.512253
std,0.003728,0.011773,12.083419,0.944495,2664.131028
min,45.464944,-73.635002,28.000000,0.000000,0.000000
25%,45.468345,-73.625974,51.750000,2.639801,2340.691893
50%,45.472427,-73.616678,63.000000,3.012348,4694.263953
75%,45.475249,-73.605759,66.000000,3.251237,6979.636979
max,45.477693,-73.595871,84.000000,6.635520,9006.744506


## Check last run date

In [10]:
head_tail = os.path.split(ll[-1])
print(head_tail[1][0:-4])

NDGRC_2024_09_16


# Save as csv

In [11]:
!ls ../data/csv/NDGRC*csv | wc -l

zsh:1: no matches found: ../data/csv/NDGRC*csv
       0


In [ ]:
!rm ../data/csv/NDGRC*csv

In [16]:
# get list of gpx file name
# -> list_all_files_gpx from above

# create list of csv file name
list_csv_file_name = [path_data_csv+os.path.split(x)[1][0:-4]+"_downSample.csv" for x in list_all_files_gpx]

# export DataFrame as csv file
for c,df_ in enumerate(list_all_files_ReSample_as_df):
    df_.to_csv(list_csv_file_name[c], index=False)
    

In [17]:
!ls ../data/csv/NDGRC*.csv | wc -l

      54


Check that the csv files has been created.

In [18]:
# load one file
new_df = pd.read_csv(path_data_csv+"NDGRC_2023_03_06_downSample.csv")

# to check if it is the same as the list of Dataframe
new_df["cumulative_distance"].iloc[-1] - list_all_files_ReSample_as_df[-1]["cumulative_distance"].iloc[-1]

-167.09175713996774

Datavisualization in another notebook.